In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import models, layers, optimizers

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df.csv')
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [3]:
df = df.drop(columns=[
    'Late_delivery_risk'
], axis=1)

In [4]:
df_obj = df.select_dtypes(include='object')
df_obj.drop(columns=['Delivery Status'], axis=1, inplace=True)

In [5]:
#one hot encoder
def onehotencoder(col):
    
    ohe = OneHotEncoder()
    ohe_array = ohe.fit_transform((pd.DataFrame(col))).toarray()

    feature_labels = np.array(ohe.categories_).ravel()


    df_obj = pd.DataFrame(ohe_array, columns=feature_labels)
    
    return df_obj

In [6]:
df_ohe = pd.DataFrame()
for c in df_obj.columns:
    one_hot = onehotencoder(df_obj[c])
    df_ohe = pd.concat([df_ohe, one_hot], axis=1) 

In [7]:
df_cont_var = df.select_dtypes(exclude='object')

In [8]:
df_ohe = pd.concat([df_cont_var, df_ohe], axis=1)

In [9]:
df_ohe = pd.concat([df_ohe, df['Delivery Status'] ], axis=1)

In [10]:
df=df_ohe

In [11]:
df.replace(to_replace=['Shipping on time'], value=0, inplace=True)
df.replace(to_replace=['Advance shipping'], value=1, inplace=True)
df.replace(to_replace=['Late delivery'], value=2, inplace=True)

In [12]:
y=df['Delivery Status']
y= to_categorical(y)

In [13]:
# x variable
X = df.drop(columns=['Delivery Status'], axis=1)

In [14]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=.3)

In [15]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=.5)

In [16]:
# another way to build model
#model = Sequential([
#    Dense(10, activaiton='relu')
#    Dense(10, activaiton='relu')
#    Dense(2, activaiton='softmax')
#])

In [17]:
model = models.Sequential()

In [18]:
model.add(layers.Dense(20, activation='relu', input_shape=(len(X.columns),)))
model.add(layers.Dense(20, activation='relu'))

model.add(layers.Dense(3, activation='softmax'))

In [19]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=20,
    validation_data=(X_val, y_val)
)

Epoch 1/50
6047/6047 [==============================] - 6s 949us/step - loss: 1403484.1250 - accuracy: 0.5716 - val_loss: 0.9694 - val_accuracy: 0.5784
Epoch 2/50
6047/6047 [==============================] - 6s 925us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.9695 - val_accuracy: 0.5784
Epoch 3/50
6047/6047 [==============================] - 5s 907us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.9694 - val_accuracy: 0.5784
Epoch 4/50
6047/6047 [==============================] - 6s 909us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.9697 - val_accuracy: 0.5784
Epoch 5/50
6047/6047 [==============================] - 6s 907us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.9694 - val_accuracy: 0.5784
Epoch 6/50
6047/6047 [==============================] - 6s 923us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.9695 - val_accuracy: 0.5784
Epoch 7/50
6047/6047 [==============================] - 6s 922us/step - loss: 0.9763 - accuracy: 0.5718 - val_loss: 0.

In [23]:
model.evaluate(X_test, y_test)

810/810 [==============================] - 1s 743us/step - loss: 0.9751 - accuracy: 0.5727


[0.9751445055007935, 0.5726799368858337]